# Split DataFrame (Statut: OK)

Le but de ici est de séparer le DF en 2 afin de le manipuler plus aisément. 

1. DF sur la participation 
1. DF sur les scores


***Import library***

In [ ]:
import pandas as pd
import numpy as np

## Import DF

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/h4r1c0t/WildCodeSchool/master/Project/Datathon%231/Datasets/bigData.csv',
                index_col=False)

In [ ]:
df.head()

## 1. Participation's DF

In [ ]:
lstCol = ['Date', 'Tour', 'Code département', 'département', 'circonscription', 
          'Inscrits', 'Votants', 'Exprimés', 'Blancs et nuls']
dfP = df[lstCol]

dfP.shape

In [ ]:
dfP = dfP.drop_duplicates()

dfP.shape

In [ ]:
tab1 = pd.pivot_table(dfP, values = ['Inscrits', 'Votants', 'Exprimés', 'Blancs et nuls'], 
                      index = ['Date', 'Tour', 'Code département', 'département'],
                      columns=[], aggfunc=np.sum)

In [ ]:
# Nb Abstention
tab1['Abstention'] = tab1['Inscrits'] - tab1['Votants']

# % Participation 
tab1['Votants (%)'] = round(tab1['Votants'] / tab1['Inscrits'] * 100, 2)

# % Abstention
tab1['Abstention (%)'] = round(tab1['Abstention'] / tab1['Inscrits'] * 100, 2)

In [ ]:
tab1.head()

In [ ]:
tab1.reset_index(inplace = True)

tab1['Année'] = tab1['Date'].apply(lambda x: x[:4])

In [ ]:
tab1.head()

In [ ]:
tab1.to_csv('/home/h4r1c0t/Github/WildCodeSchool/Project/Datathon#1/Datasets/data_participation_1965-2012.csv')

## Score's DF

In [ ]:
df.head()

In [ ]:
lstCol = ['Date', 'Tour', 'Code département', 'département', 'circonscription', 
          'Candidats', 'Exprimés', 'Score']
dfSc = df[lstCol]

orientations = pd.read_csv('https://raw.githubusercontent.com/h4r1c0t/WildCodeSchool/master/Project/Datathon%231/Datasets/orientation_candidats.csv')

dfSc = dfSc.merge(orientations, how = 'left', left_on = 'Candidats', right_on = 'Candidat')
dfSc.drop('Candidat', axis = 1, inplace = True)

dfSc.shape

In [ ]:
dfSc.head()

In [ ]:
tab2 = pd.pivot_table(dfSc, values = ['Score'], 
                      index = ['Candidats', 'Orientation', 'Date', 'Tour', 'Code département', 'département'],
                      columns = [], aggfunc = np.sum)

In [ ]:
tab2.reset_index(inplace = True)

tab2['Année'] = tab2['Date'].apply(lambda x: x[:4])

expr = tab1[['Date', 'Code département', 'Exprimés']]

tab2 = tab2.merge(expr, left_on = ['Date', 'Code département'], right_on = ['Date', 'Code département'])

In [ ]:
tab2['Score (%)'] = round(tab2['Score'] / tab2['Exprimés'] * 100, 2)
tab2.drop('Exprimés', axis = 1, inplace = True)

In [ ]:
tab3 = pd.pivot_table(tab2, values = ['Score (%)'], 
                      index = ['Date', 'Tour', 'Candidats'], 
                      aggfunc = np.mean).reset_index()

In [ ]:
tab3[tab3['Date'] == '1965-12-05']

In [ ]:
tab2.to_csv('/home/h4r1c0t/Github/WildCodeSchool/Project/Datathon#1/Datasets/data_scores_1965-2012.csv')